# Strands Agents SDK: with Bedrock Agents integration, user feedback  with LangFuse observability

## Overview
The focus of this notebook is to build a **multi agent application** using diverse agents framework — Strands, LangGraph and Amazon Bedrock..

Strands Agents have build-in support for **observability with LangFuse**. On this example we will demonstrate how to build an agent with observability and evaluation. We will leverage [Langfuse](https://langfuse.com/) to process the Strands Agent traces and [Ragas](https://www.ragas.io/) metrics to evaluate the performance of  agent. The primary focus is on agent evaluation the quality of responses generated by the Agent use the traces produced by the SDK. 


## Agent Details
<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                         |
|--------------------|----------------------------------------------------|
|Native tools used   |current_time, retrieve                              |
|Custom tools created|langgraph_web_intelligence, 5 bedrock finance agents|
|Agent Structure     |Single agent architecture supervising 6 multiple agents                          |
|AWS services used   |Amazon Bedrock Agents, Amazon EKS     |
|Integrations        |LangFuse for observability and Ragas for evaluation|

</div>

### Dependencies setup

In [1]:
%pip install ipywidgets  "strands-agents==0.1.9" "strands-agents-tools==0.1.7" "langfuse==3.1.1" -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Core LangFuse v3 imports

from langfuse import observe, get_client

import uuid
import boto3
import os
import base64
from dotenv import load_dotenv

### Enviroment variables and LangFuse connection setup

In [ ]:
## 1. Set general environment variables first

os.environ["AWS_REGION_NAME"]="us-east-1" #AWS Region

# 2. Get keys for your project from the project settings page: you can create an account on Langfuse portal for testing
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-xxxxxxxx" # Your Langfuse project secret key
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-xxxxxxxx" # Your Langfuse project public key
os.environ["LANGFUSE_HOST"] = "xxxxxxx" # Langfuse domain


# 3. Set up endpoint for OpenTelemetry with proper Basic authentication


otel_host = os.environ.get("LANGFUSE_HOST")
if otel_host:
    # Set up endpoint for OpenTelemetry
    otel_endpoint = str(os.environ.get("LANGFUSE_HOST")) + "/api/public/otel/v1/traces"
    
    # Create authentication token for OpenTelemetry using Basic auth
    auth_token = base64.b64encode(
        f"{os.environ.get('LANGFUSE_PUBLIC_KEY')}:{os.environ.get('LANGFUSE_SECRET_KEY')}".encode()
    ).decode()
    os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = otel_endpoint
    os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {auth_token}"
    
    print(f"LANGFUSE_HOST: {os.environ.get('LANGFUSE_HOST')}")
    print(f"OTEL_EXPORTER_OTLP_ENDPOINT: {os.environ.get('OTEL_EXPORTER_OTLP_ENDPOINT')}")
    print(f"Authentication configured with Basic auth")

##  Multi-framework Agent Setup

### Defining main agent system prompt and model

In [ ]:
from strands_tools import retrieve, current_time
from strands import Agent, tool
from strands.models.bedrock import BedrockModel
import boto3


## 4. Load environment variables
load_dotenv()

## 5. Create model with explicit model_id

#"""
model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0" # Model to be used by main agent 
print(f"Using model ID: {model_id}")
bedrock_model = BedrockModel(
    model_id=model_id,
    temperature=0.3,
    max_tokens=2000,
    top_p=0.8,
    guardrail_trace="enabled",           # Enable trace info for debugging
)
#"""


## 6. System Prompt / Instructions for the Strands-Agent Multi framework agent

system_prompt = '''
As a Trip Planner, you take advantage of your specialists agents 
(activity_planner, restaurant_scout, and itinerary_compiler) at planning activities and finding good restaurants. 
You also create itineraries to package all of that in a clear plan.

'''

## 5. Conversation Manager to store the last 15 conversations

from strands.agent.conversation_manager import SlidingWindowConversationManager
conv_manager = SlidingWindowConversationManager(window_size=15)

Using model ID: us.anthropic.claude-3-7-sonnet-20250219-v1:0


#### Declaring Bedrock Restaurant Scout as @Tool

In [ ]:
@tool
def RestaurantScoutAgent(query):
    print("Calling Bedrock Restaurant Scout Agent")
    
    region = "us-east-1"
    agent_id = "xxxx"
    alias_id = "xxx"
    
    print(f"Region: {region}, Agent ID: {agent_id}, Alias ID: {alias_id}")

    bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region_name=region)
    session_id = str(uuid.uuid1())
    end_session = False
    enable_trace = True

    # invoke the agent API
    try:
        agentResponse = bedrock_agent_runtime_client.invoke_agent(
            inputText=query,
            agentId=agent_id,
            agentAliasId=alias_id, 
            sessionId=session_id,
            enableTrace=enable_trace, 
            endSession=end_session,
        )
        
        event_stream = agentResponse['completion']
        agent_answer = ""
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                agent_answer += data.decode('utf8')
            else:
                print(f"Unexpected event: {event}")
        
        return agent_answer
    except Exception as e:
        print(f"Error invoking Bedrock agent: {e}")
        return f"Error: {str(e)}"

#### Declaring Bedrock Activity Finder as @Tool

In [6]:
@tool
def ActivityFinderAgent(query):
    print("CALLING BEDROCK ACTIVITY FINDER")
    
    region = "us-east-1"
    agent_id = "ULSJ8ADVZT"
    alias_id = "5DGSYX3ZYF"
    
    print(f"Region: {region}, Agent ID: {agent_id}, Alias ID: {alias_id}")

    bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region_name=region)
    session_id = str(uuid.uuid1())
    end_session = False
    enable_trace = True

    # invoke the agent API
    try:
        agentResponse = bedrock_agent_runtime_client.invoke_agent(
            inputText=query,
            agentId=agent_id,
            agentAliasId=alias_id, 
            sessionId=session_id,
            enableTrace=enable_trace, 
            endSession=end_session,
        )
        
        event_stream = agentResponse['completion']
        agent_answer = ""
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                agent_answer += data.decode('utf8')
            else:
                print(f"Unexpected event: {event}")
        
        return agent_answer
    except Exception as e:
        print(f"Error invoking Bedrock agent: {e}")
        return f"Error: {str(e)}"

#### Declaring Bedrock Itinerary Compiler Agent as @Tool

In [ ]:
@tool
def ItineraryCompilerAgent(query):
    print("CALLING BEDROCK ITINEARY COMPILER AGENT")
    
    region = "us-east-1"
    agent_id = "xxxx"
    alias_id = "xxxx"
    
    print(f"Region: {region}, Agent ID: {agent_id}, Alias ID: {alias_id}")

    bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region_name=region)
    session_id = str(uuid.uuid1())
    end_session = False
    enable_trace = True

    # invoke the agent API
    try:
        agentResponse = bedrock_agent_runtime_client.invoke_agent(
            inputText=query,
            agentId=agent_id,
            agentAliasId=alias_id, 
            sessionId=session_id,
            enableTrace=enable_trace, 
            endSession=end_session,
        )
        
        event_stream = agentResponse['completion']
        agent_answer = ""
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                agent_answer += data.decode('utf8')
            else:
                print(f"Unexpected event: {event}")
        
        return agent_answer
    except Exception as e:
        print(f"Error invoking Bedrock agent: {e}")
        return f"Error: {str(e)}"

#### Main Agent instantiation with Tracing metadata

In [8]:
tool_list = [RestaurantScoutAgent,ActivityFinderAgent,ItineraryCompilerAgent]
#Generating unique UUID for Agent Session
import uuid
session_id = str(uuid.uuid4())
print(session_id)

# Verify model_id is set correctly before creating the agent
print(f"Creating agent with model_id: {model_id}")

agent = Agent(
    model=bedrock_model,
    tools=tool_list,
    system_prompt=system_prompt,
    conversation_manager=conv_manager,
    
    trace_attributes={
        # Core Langfuse identifiers
        "session.id": session_id,
        "user.id": "palacan@amazon.com",
        "model.id": bedrock_model,
        
        # Langfuse-specific metadata
        "langfuse.tags": [
            f"Agent-TripPlanner",
            f"Agent-TripPlanner-v5",
            f"Model-{bedrock_model}",
        ],
        "langfuse.environment": "development",  # Environment label

        #Guardrails setup
        "guardrail.enabled": False,
        "guardrail.id": "unpt29yplqsx",        
        
        # Performance context
        "performance.threshold": "3000ms",  # SLA reference

        # Custom metadata (arbitrary key-value pairs)
        "metadata": {
            "device.type": "mobile-web",
            "browser.version": "Chrome/125.0.6422.112",
            "campaign.source": "google-ads",
            "ab_test.group": "variant-c"
        }
    }
)

8519e748-f4e6-4bcc-9725-938f60ba368e
Creating agent with model_id: us.anthropic.claude-3-7-sonnet-20250219-v1:0


### Test agents responses

#### Regular Agent invoke  With Amazon Bedrock Guardrails tracing

In [9]:
@observe()
def call_agent_with_guardrails(agent, prompt, trace_id=None):
    langfuse = get_client()
    
    # Call your agent
    response = agent(prompt)
    
    # Check for guardrail intervention
    if hasattr(response, 'stop_reason') and response.stop_reason == "guardrail_intervened":
        guardrail_intervened = True
        guardrail_action = "blocked"
        
        # Update the current trace
        langfuse.update_current_trace(
            metadata={
                "guardrail.intervened": guardrail_intervened,
                "guardrail.action": guardrail_action
            },
            tags=["guardrail-monitored","guardrail-blocked"]
        )
    else:
        langfuse.update_current_trace(
            metadata={
                "guardrail.intervened": False
            },
            tags=["guardrail-monitored","guardrail-passed"]
        )
    
    return response



In [10]:
# Usage

query="""
    Find highly-rated restaurants and dining experiences at {destination}.
    Use internet search tools, restaurant review sites, and travel guides.
    Make sure to find a variety of options to suit different tastes and budgets, and ratings for them.

    Traveler's information:

    - origin: Mexico City
    - destination: Medellin
    - age of the traveler: 40
    - hotel localtion: Medellin, Marriott Hotel
    - arrival: 15 December 2025
    - departure: 3 January 2026
    - food preferences: Sushi, Thai
"""

response = call_agent_with_guardrails(agent, query)

I'll help you find highly-rated restaurants and dining experiences in Medellin, Colombia, with a focus on sushi and Thai options, as well as other great dining experiences to suit different tastes and budgets.

Let me use my Restaurant Scout tool to gather this information for you:
Tool #1: RestaurantScoutAgent
Calling Bedrock Restaurant Scout Agent
Region: us-east-1, Agent ID: CD5T9SHCNP, Alias ID: K0SGAE62WN
Unexpected event: {'trace': {'agentAliasId': 'K0SGAE62WN', 'agentId': 'CD5T9SHCNP', 'agentVersion': '1', 'callerChain': [{'agentAliasArn': 'arn:aws:bedrock:us-east-1:657000049104:agent-alias/CD5T9SHCNP/K0SGAE62WN'}], 'sessionId': '2c354e54-9f1a-11f0-bd5c-a21e6fd8ec27', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'foundationModel': 'anthropic.claude-3-5-sonnet-20240620-v1:0', 'inferenceConfiguration': {'maximumLength': 2048, 'stopSequences': ['</invoke>', '</answer>', '</error>'], 'temperature': 0.0, 'topK': 250, 'topP': 1.0}, 'text': '{"system":" Role: Restaurant Sc

In [ ]:
# Usage

query="""
Compile all researched information into a comprehensive day-by-day itinerary for the trip to Medellin.
    Ensure the itinerary integrates all planned activities and dining experiences.
    Use text formatting and document creation tools to organize the information.
"""

response = call_agent_with_guardrails(agent, query)

### Trace ID capture from last Agent query  

In [ ]:
if agent.trace_span:
    span_context = agent.trace_span.get_span_context()
    trace_id = format(span_context.trace_id, "032x")
    print(f"Extracted trace_id: {trace_id}")

### User feedback tracking on LangFuse

Simulating a user feedback for register the feedback on last trace

In [12]:
import ipywidgets as widgets
from IPython.display import display

def send_feedback(trace_id: str, value: int):
    langfuse.create_score(
        trace_id=trace_id,
        name="user_feedback",
        value=value
    )
    print(f"Feedback submitted: {'👍' if value == 1 else '👎'}")

def send_comment(trace_id: str, comment: str):
    langfuse.create_score(
        trace_id=trace_id,
        value=1,
        name="user_feedback_comment",
        metadata=comment
    )
    print("Comment submitted!")

def create_feedback_ui(trace_id: str):
    # Feedback question and buttons
    question = widgets.HTML("<b>Was the response helpful?</b>")
    btn_up = widgets.Button(description="👍", tooltip="Positive feedback")
    btn_down = widgets.Button(description="👎", tooltip="Negative feedback")
    
    # Comments section
    comments_label = widgets.HTML("<b>Please provide feedback to help us improve</b>")
    comments_text = widgets.Textarea(
        placeholder='Type your feedback here...',
        layout=widgets.Layout(width='60%', height='50px')
    )
    submit_comment_btn = widgets.Button(description="Submit Comment", button_style='info')
    
    # Button handlers
    def on_up_click(_):
        send_feedback(trace_id, 1)
    
    def on_down_click(_):
        send_feedback(trace_id, 0)
        
    def on_comment_submit(_):
        comment = comments_text.value.strip()
        if comment:
            send_comment(trace_id, comment)
            comments_text.value = ""  # Clear after submit
    
    btn_up.on_click(on_up_click)
    btn_down.on_click(on_down_click)
    submit_comment_btn.on_click(on_comment_submit)
    
    # Layout
    feedback_box = widgets.VBox([question, widgets.HBox([btn_up, btn_down])])
    comments_box = widgets.VBox([comments_label, comments_text, submit_comment_btn])
    display(widgets.VBox([feedback_box, comments_box]))

# Example usage
create_feedback_ui(trace_id)
